# Calculate Urban Metrics: Shape

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import time

In [ ]:
import pandas as pd
import geopandas as gpd
import math

In [ ]:
# Get reference to GOSTNets
sys.path.append(r"C:\repos\INFRA_SAP")
from infrasap.urban_metrics import *

In [ ]:
start_time = time.time()

## To-do
### Calculate for Central Asia
Kazakhstan, Kyrgystan, Tajikistan, Afghanistan, Turkmenistan, and Uzbekistan
### Calculate for ECA (https://www.hurriyetdailynews.com/world-bank-updates-gdp-forecast-for-europe-central-asia-158933)
Afghanistan, Albania, Armenia, Azerbaijan, Belarus, Bosnia and Herzegovina, Bulgaria, Croatia, Georgia, Kazakhstan, Kosovo, Kyrgyzstan, Moldova, Montenegro, North Macedonia, Poland, Romania, Serbia, Tajikistan, Turkey, Turkmenistan, Ukraine, Russia, Hungry, and Uzbekistan.

In [ ]:
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\russia_urban_extents_merged_4326.shp"

# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\sample_shps_3.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UBZ_only_FUAs2_geom_fixed.shp"
# shpName = r"C:\repos\GOST_Urban\Notebooks\Implementations\eca_wo_rus_urban_clusters_ghs_pop_smooth_100k_4326_2.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_4326_geom_fixed_greater_50k.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_4326_geom_fixed.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\ECA_wo_rus_urban_extents\eca_wo_rus_built_up_extents_4326.shp"
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_clusters_5k_up_4326.shp"

In [ ]:
# import extent
input_shapes_gpd = gpd.read_file(shpName)

In [ ]:
# input_shapes_gpd

In [ ]:
# proj
# vars(input_shapes_gpd)
input_shapes_gpd._crs

In [ ]:
# make the GeoDataFrame unprojected
input_shapes_gpd = input_shapes_gpd.to_crs("epsg:4326")
input_shapes_gpd._crs

In [ ]:
input_shapes_gpd

In [ ]:
start_time = time.time()

In [ ]:
# -------------------------------------
# SET UP TEMP WORKSPACE...
TempWS = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\Shape_Metrics_Temp"

# feature grid text file...
gridTxtFile = "%s\\grid.txt" % TempWS

### Loop through each urban extent and calculate its urban metrics. For each urban extent we will reproject it using the UTM zone it belongs in. Then at the end we will unproject it again.

In [ ]:
%%time
for index, row in input_shapes_gpd.iterrows():
    print(index)
    metrics = {}

    # creates a temporary GDF for just the row's shape
    temp_gdf = input_shapes_gpd.iloc[[index]]

    # finds its correct UTM zone projection and reprojects it
    temp_gdf_proj = project_gdf(temp_gdf)

    A = temp_gdf_proj.iloc[0].geometry.area
    P = temp_gdf_proj.iloc[0].geometry.length

    # Equal area circle radius...
    r = (
        (temp_gdf_proj.iloc[0].geometry.area / math.pi) ** 0.5
    )  # radius of equal area circle (circle with area equal to shape area) (derived from A = pi * r squared)
    print(f"print r: {r}")
    p = 2 * math.pi * r  # Equal area circle perimeter

    # LIST OF COORDINATES OF FEATURE VERTICES (for single part features)...
    pntLst = []  # stores feature array...
    subVLst = []

    # Step through exterior part of the feature
    for coord in temp_gdf_proj.iloc[0].geometry.exterior.coords:
        # Print the part number
        # print("coord {}:".format(coord))
        # Step through each vertex in the feature
        # Print x,y coordinates of current point
        # print("{}, {}".format(coord[0], coord[1]))
        X, Y = coord[0], coord[1]  # get point XY
        subVLst.append([X, Y])  # add XY to list

    pntLst.append(subVLst)

    # if it has interior polygons
    if len(list(temp_gdf_proj.iloc[0].geometry.interiors)) > 0:
        for poly in list(temp_gdf_proj.iloc[0].geometry.interiors):
            print("new interior polygon")
            subVLst = []
            # Step through each part of the feature
            for coord in poly.coords:
                # print("coord {}:".format(coord))
                # Step through each vertex in the feature
                # Print x,y coordinates of current point
                # print("{}, {}".format(coord[0], coord[1]))
                X, Y = coord[0], coord[1]  # get point XY
                subVLst.append([X, Y])  # add XY to list
            # print(subVLst)
            subVLst.reverse()
            # print(subVLst)
            pntLst.append(subVLst)

    # desired shape area in pixels...
    numPix = 20000

    # calculate pixel size...
    cellsize = (A / numPix) ** 0.5

    # get min and max XY values
    minX, minY, maxX, maxY = (
        temp_gdf_proj.iloc[0].geometry.bounds[0],
        temp_gdf_proj.iloc[0].geometry.bounds[1],
        temp_gdf_proj.iloc[0].geometry.bounds[2],
        temp_gdf_proj.iloc[0].geometry.bounds[3],
    )

    # offset grid by half a pixel...
    minX -= cellsize / 2
    maxY += cellsize / 2

    # centroid coordinates
    centroidXY = (
        temp_gdf_proj.iloc[0].geometry.centroid.x,
        temp_gdf_proj.iloc[0].geometry.centroid.y,
    )
    x_offset, y_offset = 0, 0
    Xc, Yc = centroidXY[0] - x_offset, centroidXY[1] - y_offset

    # generates a list of points within the shape
    featPntLst = generate_featPntLst(
        pntLst, minX, minY, maxX, maxY, cellsize, gridTxtFile
    )

    # NOTE: THE CENTROID IS CURRENTLY USED AS THE CENTER
    # calculate distance of feature points to center...
    D_to_Center, EAC_pix = proximity(featPntLst, Xc, Yc, r)

    # Proximity index (circle / shape)
    # avg distance to center for equal area circle...
    circD = r * (2.0 / 3.0)
    # print(f"print circD: {circD}")
    # print(f"print D_to_Center: {D_to_Center}")
    ProximityIndex = circD / D_to_Center
    metrics["ProximityIndex"] = ProximityIndex

    # Roundness (exchange-index)
    inArea = EAC_pix * cellsize**2
    areaExchange = inArea / A
    metrics["RoundnessIndex"] = areaExchange

    # Cohesion index
    # custom tool calculates approx. average interpoint distances between
    # samples of points in shape...
    shp_interD = interpointDistance(featPntLst)

    # average interpoint distance for equal area circle...
    circ_interD = r * 0.9054

    # cohesion index is ratio of avg interpoint distance of circle to
    # avg interpoint distance of shape...
    CohesionIndex = circ_interD / shp_interD

    metrics["CohesionIndex"] = CohesionIndex

    # Spin index
    # custom tool calculates moment of inertia for shape...
    shpMOI = spin(featPntLst, Xc, Yc)

    # moment of inertia for equal area circle...
    circ_MOI = 0.5 * r**2

    # calculate spin index (circle / shape)...
    Spin = circ_MOI / shpMOI

    metrics["SpinIndex"] = Spin

    # Perimeter index (circle / shape)
    PerimIndex = p / P  # The Perimeter Index
    metrics["PerimIndex"] = PerimIndex

    # Pre-calculations for Depth, Girth, and Dispersion indices

    # print(f"print first 3 of pntLst: {pntLst[0][:3]}")

    # get list of points evenly distributed along perimeter...
    perimPntLst = PerimeterPnts(pntLst, 500)

    # print(f"print first of perimPntLst: {perimPntLst[0]}")

    # ------------------------------------------------------------------------------
    # SECTION 7: CALCULATE DISTANCE OF INTERIOR SHAPE POINTS TO PERIMETER POINTS...

    # custom tool calculates distance of each interior point to nearest perimeter point...
    pt_dToE = pt_distToEdge(featPntLst, perimPntLst)

    # print(f"print max pt_dToE: {pt_dToE[-1]}")

    # Depth index
    # custom tool calculates average distance from interior pixels to nearest edge pixels...
    shp_depth = depth(pt_dToE)

    # depth for equal area circle...
    EAC_depth = r / 3

    # calculate depth index (shape / circle)...
    depthIndex = shp_depth / EAC_depth
    metrics["DepthIndex"] = depthIndex

    # Girth index
    # custom tool calculates shape girth (distance from edge to innermost point)
    # and outputs list position of innermost point...
    shp_Girth = girth(pt_dToE)

    # calculate girth index (shape / circle)...
    girthIndex = shp_Girth / r  # girth of a circle is its radius
    # print(f"print shp_Girth: {shp_Girth}")
    # print(f"print r: {r}")
    metrics["GirthIndex"] = girthIndex

    # Dispersion index
    # custom tool calculates average distance between proximate center and edge points...
    dispersionIndex, avgD = dispersion([Xc, Yc], perimPntLst[0])
    metrics["DispersionIndex"] = dispersionIndex

    # Detour index
    # custom tool creates list of points in the exterior polygon shape
    hullPerim = ConvexHull(pntLst[0])

    # calculate detour index (circle / shape)...
    detourIndex = p / hullPerim
    metrics["DispersionIndex" "DispersionIndex"] = detourIndex

    # Range index
    # custom tool identifies perimeter points that are farthest apart, outputs
    # distance between furthermost points...
    circumCircD = Range(pntLst[0])

    # calculate range index (circle / shape)
    rangeIndex = (2 * r) / circumCircD
    metrics["RangeIndex"] = rangeIndex

    # Put all metrics in a DataFrame
    metrics_scalar = {}
    for k in metrics:
        metrics_scalar[k] = [metrics[k]]
    metrics_df = pd.DataFrame(metrics_scalar)

    # and concatinate it with the row's shape
    new_temp_gdf_proj = pd.concat(
        [temp_gdf_proj.reset_index(drop=True), metrics_df], axis=1
    )

    # print("print new_temp_gdf_proj")
    # print(new_temp_gdf_proj)

    # make it unprojected
    temp_gdf_proj_4326 = new_temp_gdf_proj.to_crs("epsg:4326")

    # put the results of each row into a new DataFrame
    if index == 0:
        # print("creating output_shapes_gpd_4326")
        output_shapes_gpd_4326 = temp_gdf_proj_4326
    else:
        # print(f"print output_shapes_gpd_4326, and index is {index}")
        # print(output_shapes_gpd_4326)
        # print("to append temp_gdf_proj_4326")
        # print(temp_gdf_proj_4326)
        output_shapes_gpd_4326 = output_shapes_gpd_4326.append(
            temp_gdf_proj_4326, ignore_index=True
        )
        # print("output_shapes_gpd_4326 after append")
        # print(output_shapes_gpd_4326)

In [ ]:
output_shapes_gpd_4326

In [ ]:
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_metrics_results_russia"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results_all"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus_all"
output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_extents_results_5k_up"

In [ ]:
# save as CSV
# output_shapes_gpd_4326.to_csv(output + r"\ECA_russia_urban_metrics_100k_shape.csv")
# output_shapes_gpd_4326.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_shape.csv")
# output_shapes_gpd_4326.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_all_shape.csv")
output_shapes_gpd_4326.to_csv(output + r"\all_urban_metrics_5k_up_shape.csv")

In [ ]:
print(f"total time to process: {time.time()-start_time}")